In [ ]:
from models.model import ResNet50, ViTBase, CLIPModel
from models.utils import train_step, eval_step, DataLoaders
from models.transforms import transforms_resnet, transforms_vit
import torch
import torch.nn as nn
import torch
from torchvision import datasets, transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision.models import resnet18
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

### ResNet 50

In [ ]:
# Set up image transformations (adjust as needed)
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Resize all images to the same size
    transforms.ToTensor(),          # Convert to Tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalization
])

# Load dataset from folders
train_dataset = datasets.ImageFolder(root='./train', transform=transform)
test_dataset = datasets.ImageFolder(root='./texture/cue-conflict', transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)


In [ ]:
BATCH_SIZE = 64
N_EPOCHS = 3

In [ ]:
# Train and then Evaluate (Three Different Train and Evaluation Loops)

def train_and_eval(train_loader,test_loader,model,loss_fn,optimizer,device,modeltype):
    tr_metric = {"Accuracy":[],"Loss":[]}
    ts_metric = {"Accuracy":[],"Loss":[]}

    for epoch in tqdm(range(N_EPOCHS)):
        tr_loss, tr_acc = train_step(model,train_loader,loss_fn,optimizer,device,modeltype, data="texture")
        ts_loss, ts_acc = eval_step(model,test_loader,loss_fn,device,modeltype,data="texture")

        tr_metric["Accuracy"].append(tr_acc)
        tr_metric["Loss"].append(tr_loss)

        ts_metric["Accuracy"].append(ts_acc)
        ts_metric["Loss"].append(ts_loss)
    
    return tr_metric, ts_metric

model = ResNet50(n_classes=10)
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())
device = 'cuda' if torch.cuda.is_available() else 'cpu'


tr,ts = train_and_eval(train_loader,test_loader,model,loss_func,optimizer,device,'resnet50')

print("Final Train Accuracy:",tr["Accuracy"][-1])
print("Final Test Accuracy:",ts["Accuracy"][-1])
print("Final Train Loss:",tr["Loss"][-1])
print("Final Test Loss:",ts["Loss"][-1])

100%|██████████| 3/3 [04:09<00:00, 83.19s/it]

Final Train Accuracy: 0.95375
Final Test Accuracy: 0.37625
Final Train Loss: 0.35071394562721253
Final Test Loss: 1.7723349618911743


In [ ]:
import pandas as pd
pd.DataFrame(tr).to_csv("resnet50_newdataset_tr_1.csv") 
pd.DataFrame(ts).to_csv("resnet50_newdataset_ts_1.csv")

In [ ]:
torch.save(model.state_dict(),"resnet50_newdataset.pth")
model.load_state_dict(torch.load("resnet50_newdataset.pth"))   

<All keys matched successfully>

### ViT

In [ ]:
from models.model import ViTBase
from models.utils import train_step, eval_step, DataLoaders
from models.transforms import transforms_resnet
import torch
import torch.nn as nn
from torchvision.datasets import CIFAR10

In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchvision.models import resnet18
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

# Set up image transformations (adjust as needed)
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize all images to the same size
    transforms.ToTensor(),          # Convert to Tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalization
])

# Load dataset from folders
train_dataset = datasets.ImageFolder(root='./train', transform=transform)
test_dataset = datasets.ImageFolder(root='./texture/cue-conflict', transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)


In [ ]:
BATCH_SIZE = 64
N_EPOCHS = 2

In [ ]:
# Train and then Evaluate (Three Different Train and Evaluation Loops)
def train_and_eval(train_loader,test_loader,model,loss_fn,optimizer,device,modeltype):
    tr_metric = {"Accuracy":[],"Loss":[]}
    ts_metric = {"Accuracy":[],"Loss":[]}

    for epoch in tqdm(range(N_EPOCHS)):
        tr_loss, tr_acc = train_step(model,train_loader,loss_fn,optimizer,device,modeltype, data="texture")
        ts_loss, ts_acc = eval_step(model,test_loader,loss_fn,device,modeltype,data="texture")

        tr_metric["Accuracy"].append(tr_acc)
        tr_metric["Loss"].append(tr_loss)

        ts_metric["Accuracy"].append(ts_acc)
        ts_metric["Loss"].append(ts_loss)
    
    return tr_metric, ts_metric

model = ViTBase(n_classes=10)
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())
device = 'cuda' if torch.cuda.is_available() else 'cpu'


tr,ts = train_and_eval(train_loader,test_loader,model,loss_func,optimizer,device,'vitbase')

print("Final Train Accuracy:",tr["Accuracy"][-1])
print("Final Test Accuracy:",ts["Accuracy"][-1])
print("Final Train Loss:",tr["Loss"][-1])
print("Final Test Loss:",ts["Loss"][-1])

100%|██████████| 2/2 [15:06<00:00, 453.19s/it]

Final Train Accuracy: 0.99125
Final Test Accuracy: 0.44375
Final Train Loss: 0.12436148136854172
Final Test Loss: 1.7933626294136047


In [ ]:
import pandas as pd
pd.DataFrame(tr).to_csv("vitbase_newdataset_tr_1.csv") 
pd.DataFrame(ts).to_csv("vitbase_newdataset_ts_1.csv")

In [ ]:
torch.save(model.state_dict(),"vitbase_newdataset.pth")
model.load_state_dict(torch.load("vitbase_newdataset.pth"))   

<All keys matched successfully>

### CLIP

In [ ]:
# Load dataset from folders
train_dataset = ImageFolder(root='./train')
test_dataset = ImageFolder(root='./texture/cue-conflict')

# Create data loaders
loader = DataLoaders(train_dataset,test_dataset= test_dataset, batch_size=64, model_type='clip',data='texture',shuffle=True)
train_loader, test_loader = loader.get_loaders()

In [ ]:
BATCH_SIZE = 64
N_EPOCHS = 3

In [ ]:
# Train and then Evaluate (Three Different Train and Evaluation Loops)
def train_and_eval(train_loader,test_loader,model,loss_fn,optimizer,device,modeltype):
    tr_metric = {"Accuracy":[],"Loss":[]}
    ts_metric = {"Accuracy":[],"Loss":[]}

    for epoch in tqdm(range(N_EPOCHS)):
        tr_loss, tr_acc = train_step(model,train_loader,loss_fn,optimizer,device,modeltype, data="texture")
        ts_loss, ts_acc = eval_step(model,test_loader,loss_fn,device,modeltype,data="texture")

        tr_metric["Accuracy"].append(tr_acc)
        tr_metric["Loss"].append(tr_loss)

        ts_metric["Accuracy"].append(ts_acc)
        ts_metric["Loss"].append(ts_loss)
    
    return tr_metric, ts_metric

model = CLIPModel(256)
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())
device = 'cuda' if torch.cuda.is_available() else 'cpu'


tr,ts = train_and_eval(train_loader,test_loader,model,loss_func,optimizer,device,'clip')

print("Final Train Accuracy:",tr["Accuracy"][-1])
print("Final Test Accuracy:",ts["Accuracy"][-1])
print("Final Train Loss:",tr["Loss"][-1])
print("Final Test Loss:",ts["Loss"][-1])

100%|██████████| 3/3 [00:12<00:00,  4.13s/it]

Final Train Accuracy: 0.96625
Final Test Accuracy: 0.36124998
Final Train Loss: 0.5304164258472156
Final Test Loss: 4.328795304665198


In [ ]:
import pandas as pd
pd.DataFrame(tr).to_csv("clip_newdataset_tr_1.csv") 
pd.DataFrame(ts).to_csv("clip_newdataset_ts_1.csv")

In [ ]:
torch.save(model.state_dict(),"clip_newdataset.pth")
model.load_state_dict(torch.load("clip_newdataset.pth"))   

<All keys matched successfully>